### Get stock prices data from polygon API

In [1]:
#!pip install -r requirements.txt

In [2]:
from stockify import * 
import datetime as dt 
from polygon import RESTClient
from polygon.rest import models
from http import HTTPStatus
import time
    
API_key = '4Usv4WtFIeRsmpSPEsh3Kn8C2la841d9'
API_keys =['1Rwa2pddBbwCUgqzr9UQ4hZoA4SMnPSq','PuJHla6_U_GDHcNE3XSpk7L2gCCpdJ9J','4Usv4WtFIeRsmpSPEsh3Kn8C2la841d9']
client = RESTClient(API_key)

In [3]:
## Put the API key (if we do pay) into ENV_VARS
current_date = dt.datetime.now().date()
stocks_tickers = get_list_stocks_tickers(text_file_name='stocks_list',
                                         only_stock_tickers=True)

stocks_requirments = get_list_stocks_tickers(text_file_name='stocks_list',
                                         only_stock_tickers=False)


HTTP_summary = get_HTTP_status_codes_by_sequencing_get_calls(API_keys=API_keys,
                                                             stocks_tickers=stocks_tickers,
                                                             time_sleep=8)

stocks_ticker_GET = get_stock_list_GET_calls(HTTP_summary)

prices_dfs = []
dividends_dfs = []
DP_ratio_dfs = []
prices_dfs_final=[]
dividends_dfs_final=[]

time.sleep(60)

for st in stocks_ticker_GET:

            print(st)
            
            p_raw_response = get_p_process_by_ticker(API_key,
                                                client,
                                                st,
                                                get_daily_time_buckets(max_lookback_years=2,
                                                                        current_is_timestamp=False,
                                                                        frequency='daily',
                                                                        include_weekend=False),
                                                stock_prices_frequency= 'daily')
            
            pdf = get_jsonList_prices_into_dataframe(get_serialized_price_raw_into_json(p_raw_response,st),
                                        ['close'])


            prices_dfs.append(pdf)
    
            d_raw_response = get_top_x_dividends_by_ticker(ticker=st,
                                                        client=client,
                                                        API_latency_secs=1.5,
                                                        n_API_calls=24
                                                        )
            

            ddf = get_jsonList_dividends_into_dataframe(DivjsonList = d_raw_response,
                                                        measurable_time_variables= ['pay_date'],
                                                        dividend_types=['CD'])
                        
            ##SC = special dividens; all the others paid out are ordinary are marked as CD by the API.
        
            dividends_dfs.append(ddf)
            
            DP_ratio_df = get_dividend_price_ratios(ticker=st,
                                                    prices_df=pdf,
                                                    dividends_df=ddf)
        
            DP_ratio_dfs.append(DP_ratio_df)

if len(prices_dfs)>0: 
    prices_dfs_final = pd.concat(prices_dfs)
    
if len(dividends_dfs)>0: 
    dividends_dfs_final = pd.concat(dividends_dfs)
    
if len(DP_ratio_dfs)>0: 
    DP_ratio_dfs_final = pd.concat(DP_ratio_dfs)

['SBB-B', 200, 'DELAYED', 0]
['CORE-Ddssd', 200, 'DELAYED', 0]
['----', 200, 'DELAYED', 0]
ARR
AGNC
SCM
AAPL


In [5]:
DP_ratio_dfs_final[DP_ratio_dfs_final['converted_utc_timestamp']=='2022-12-30 05:00:00']

,stock,close,converted_utc_timestamp,n_paid_dividens_current_year,sum_paid_dividens_current_year,DP_ratio
0,ARR,5.63,2022-12-30 05:00:00,12.0,1.2000,21.314387
0,AGNC,10.35,2022-12-30 05:00:00,11.0,1.3200,12.753623
0,SCM,13.26,2022-12-30 05:00:00,13.0,1.2129,9.147059
0,AAPL,129.93,2022-12-30 05:00:00,4.0,0.9100,0.700377
